In [ ]:
# !pip install koreanize-matplotlib

In [ ]:
import pandas as pd
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
# 데이터 컬럼 정보를 불러옵니다.
df_col = pd.read_csv("data/kadx-utf8-recipe-detail.csv")
df_col.shape

In [ ]:
df_col

In [ ]:
# 원본 데이터를 불러옵니다.
raw = pd.read_csv("data/kadx-utf8-recipe.csv")
raw.shape

In [ ]:
# 영문 컬럼명을 한글로 변환하기 위한 매핑 딕셔너리를 생성합니다.
col_dict = df_col[["nameEn","nameKo"]].set_index("nameEn")['nameKo'].to_dict()
col_dict

In [ ]:
# 원본 데이터의 컬럼명을 한글로 변환합니다.
df = raw.rename(columns=col_dict)

In [ ]:
# 변환된 데이터의 첫 몇 줄을 확인합니다.
df.head()

In [ ]:
# 기본 EDA
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
# 시각화: 조회수, 스크랩수에 대한 분포
sns.histplot(df['조회수'], kde=True)
plt.title('조회수 분포')
plt.show()

sns.histplot(df['스크랩수'], kde=True)
plt.title('스크랩수 분포')
plt.show()

In [ ]:
# 결측치 처리
df['요리인분명'].fillna('미정', inplace=True)
df['요리시간명'].fillna('미정', inplace=True)

# 요리 인기도 분석 (조회수, 추천수, 스크랩수)
df['인기도'] = df['조회수'] + df['추천수'] + df['스크랩수']
popular_recipes = df.sort_values(by='인기도', ascending=False)

# 인기도 상위 5개 레시피
top_5_popular = popular_recipes[['레시피제목', '요리명', '조회수', '추천수', '스크랩수', '인기도']].head()
display("인기도 상위 5개 레시피:\n", top_5_popular)

# 요리 방법, 상황, 재료별 인기도 분석
def plot_popularity_by_feature(feature):
    plt.figure(figsize=(10, 3))
    sns.barplot(x=feature, y='인기도', data=df, estimator=sum)
    plt.title(f'{feature}별 레시피 인기도')
    plt.xticks(rotation=45)
    plt.show()

# 요리방법, 요리상황, 요리재료별 인기도 시각화
for feature in ['요리방법별명', '요리상황별명', '요리재료별명']:
    plot_popularity_by_feature(feature)

# 요리 난이도 및 시간별 인기도 분석
plot_popularity_by_feature('요리난이도명')
plot_popularity_by_feature('요리시간명')

# 인사이트 도출
# 1. 특정 요리방법이나 요리상황에서 인기가 높은 레시피 파악
# 2. 요리 난이도와 시간이 인기에 미치는 영향 분석
# 3. 인기 있는 레시피를 기반으로 메뉴 구성 및 프로모션 전략 수립

# 추가 분석: 자영업자에게 유용한 추천 레시피
# 특정 조건에 따라 추천할 레시피를 필터링합니다 (예: 빠르게 조리할 수 있고 인기가 높은 레시피)
filtered_recipes = df[(df['요리시간명'] != '미정') & (df['인기도'] > df['인기도'].mean())]
display("추천 레시피:\n", filtered_recipes[['레시피제목', '요리명', '요리시간명', '인기도']])

In [ ]:
pd.crosstab(df['요리상황별명'], df['요리방법별명'])

In [ ]:
pd.crosstab(df['요리상황별명'], df['요리재료별명'])

In [ ]:
pd.crosstab(df['요리방법별명'], df['요리재료별명'])

In [ ]:
# 텍스트 분석: 요리소개와 재료내용을 TF-IDF로 변환
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['요리소개'])

# 군집 분석: KMeans 사용
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(tfidf_matrix)
df['Cluster'] = kmeans.labels_

# 군집 분석 결과 시각화 (PCA로 차원 축소)
pca = PCA(n_components=2)
pca_components = pca.fit_transform(tfidf_matrix.toarray())
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=df['Cluster'])
plt.title('KMeans Clustering of Recipes')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.show()

# 군집별 주요 키워드 나열
def get_top_keywords(tfidf_matrix, kmeans, vectorizer, n_terms=10):
    df_top_keywords = pd.DataFrame()
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    
    for i in range(kmeans.n_clusters):
        top_keywords = [terms[ind] for ind in order_centroids[i, :n_terms]]
        df_top_keywords[f'Cluster {i}'] = top_keywords
    
    return df_top_keywords

# 군집별 주요 키워드 출력
df_keywords = get_top_keywords(tfidf_matrix, kmeans, vectorizer)
print("\n군집별 주요 키워드:\n", df_keywords)

# 군집별 레시피 개수
print("\n군집별 레시피 개수:\n", df['Cluster'].value_counts())

In [ ]:
# 동그라미 마스크 생성
x, y = np.ogrid[:800, :800]
center = 400
radius = 400
circle_mask = np.ones((800, 800), dtype=np.uint8) * 255
mask = (x - center) ** 2 + (y - center) ** 2 > radius ** 2
circle_mask[mask] = 0
plt.imshow(circle_mask)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from PIL import Image
from matplotlib import font_manager, rc

# 한글 폰트 설정 (맥OS 기준, 윈도우의 경우 적절한 폰트 경로로 변경 필요)
font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

# 원형 마스크 생성
def create_circle_mask(size):
    x, y = np.ogrid[:size, :size]
    center = (size - 1) / 2
    circle_mask = (x - center) ** 2 + (y - center) ** 2 > (center ** 2)
    mask = np.ones((size, size), dtype=np.uint8) * 255
    mask[circle_mask] = 0
    return mask

# 워드 클라우드 생성 함수 (서브플롯을 활용)
def create_wordcloud_subplots(df_keywords, kmeans, vectorizer):
    fig, axes = plt.subplots(1, 3, figsize=(20, 20))  # 2x2 서브플롯 생성
    axes = axes.flatten()
    
    circle_mask = create_circle_mask(800)  # 800x800 크기의 원형 마스크 생성
    
    for i in range(kmeans.n_clusters):
        cluster_keywords = df_keywords[f'Cluster {i}']
        cluster_center = kmeans.cluster_centers_[i]
        keyword_weights = [cluster_center[vectorizer.vocabulary_[keyword]] for keyword in cluster_keywords]
        
        wordcloud = WordCloud(font_path=font_path,
                              background_color='white',
                              mask=255-circle_mask,
                              contour_width=0,  # 외곽선 두께를 0으로 설정하여 원 바깥의 선을 없앰
                              width=800,
                              height=800,
                              max_font_size=150,
                              min_font_size=10).generate_from_frequencies(dict(zip(cluster_keywords, keyword_weights)))
        
        axes[i].imshow(wordcloud, interpolation='bilinear')
        axes[i].axis('off')
        axes[i].set_title(f'Cluster {i} Word Cloud', fontsize=20)
    
    plt.tight_layout(pad=0)
    plt.show()

# 전체 데이터에 대한 워드 클라우드 생성에 필요한 키워드와 가중치를 계산하는 함수
def get_overall_keywords_and_weights(tfidf_matrix, vectorizer):
    overall_weights = tfidf_matrix.sum(axis=0).A1  # 각 단어의 전체 가중치를 계산
    keywords = vectorizer.get_feature_names_out()
    keyword_weights = dict(zip(keywords, overall_weights))
    
    sorted_keywords = sorted(keyword_weights.items(), key=lambda item: item[1], reverse=True)
    sorted_keywords = sorted_keywords[:50]  # 상위 50개 키워드 선택
    all_keywords, all_weights = zip(*sorted_keywords)
    
    return all_keywords, all_weights

# get_top_keywords 함수를 사용하여 각 군집의 주요 키워드 추출
df_keywords = get_top_keywords(tfidf_matrix, kmeans, vectorizer, n_terms=50)  # 상위 50개 키워드 추출

# 서브플롯을 사용하여 각 군집별 워드 클라우드 생성
create_wordcloud_subplots(df_keywords, kmeans, vectorizer)

# 전체 데이터에 대한 키워드와 가중치 추출
all_keywords, all_weights = get_overall_keywords_and_weights(tfidf_matrix, vectorizer)

# 전체 데이터에 대한 워드 클라우드 생성 (비교용)
plt.figure(figsize=(10, 10))
circle_mask = create_circle_mask(800)  # 800x800 크기의 원형 마스크 생성
overall_wordcloud = WordCloud(font_path=font_path,
                              background_color='white',
                              mask=255-circle_mask,
                              contour_width=0,  # 외곽선 두께를 0으로 설정하여 원 바깥의 선을 없앰
                              width=800,
                              height=800,
                              max_font_size=150,
                              min_font_size=10).generate_from_frequencies(dict(zip(all_keywords, all_weights)))

plt.imshow(overall_wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Overall Word Cloud', fontsize=20)
plt.tight_layout(pad=0)
plt.show()
